In [1]:
# import utility packages
from tqdm import tqdm
from time import time

In [2]:
# import torch packages
import torch
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
import torchvision

In [3]:
# import torch summanry (like model.summary() in keras)
from torchsummary import summary

In [4]:
# import einops packages
from einops.layers.torch import Rearrange

In [5]:
# Setting hyperparameters
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device {}'.format(device))
n_epochs = 3
batch_size_train = 64
batch_size_test = 64
learning_rate = 0.01
momentum = 0.5
log_interval = 10
if device == 'cuda':
    random_seed = 42
    torch.backends.cudnn.enabled = True
    torch.manual_seed(random_seed)
    torch.cuda.manual_seed(random_seed)
else:
    random_seed = 42
    torch.backends.cudnn.enabled = False
    torch.manual_seed(random_seed)

Using device cpu


In [6]:
# Loading data
train_loader = torch.utils.data.DataLoader(
    torchvision.datasets.MNIST('files/', train=True, download=True,
                               transform=torchvision.transforms.Compose([
                                   torchvision.transforms.ToTensor(),
                                   torchvision.transforms.Normalize(
                                       (0.1307,), (0.3081,))
                               ])),
    batch_size=batch_size_train, shuffle=True)

test_loader = torch.utils.data.DataLoader(
    torchvision.datasets.MNIST('files/', train=False, download=True,
                               transform=torchvision.transforms.Compose([
                                   torchvision.transforms.ToTensor(),
                                   torchvision.transforms.Normalize(
                                       (0.1307,), (0.3081,))
                               ])),
    batch_size=batch_size_test, shuffle=True)

In [7]:
# Traditional model using Pytorch API
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 1024)
        self.fc2 = nn.Linear(1024, 10)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return F
        .log_softmax(x)

In [16]:
from layers import EinConv2d, EinMaxPool2d, EinLinear
from layers import PureEinConv2d

# My model using alternative einsum layers
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = PureEinConv2d(1, 10, kernel_size=5)
        self.conv2 = PureEinConv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = EinLinear(320, 1024)
        self.fc2 = EinLinear(1024, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = EinMaxPool2d(2)(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = self.conv2_drop(x)
        x = EinMaxPool2d(2)(x)
        x = F.relu(x)
        x = Rearrange('b c h w -> b (c h w)')(x)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return F.log_softmax(x)

In [18]:
# Trainer
def train(epoch, total_epoch):
    network.to(device).train()
    print('Train Epoch {} of {}'.format(epoch, total_epoch))
    t = tqdm(train_loader)
    for batch_idx, (data, target) in enumerate(t):
        data = data.to(device)
        target = target.to(device)
        optimizer.zero_grad()
        output = network(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % log_interval == 0:
            train_losses.append(loss.item())
            t.set_postfix({'Loss': sum(train_losses)/len(train_losses)})
            train_counter.append(batch_idx*64 + (epoch + 1)
                                 * len(train_loader.dataset))
    torch.save(network.state_dict(), 'result/model.pth')
    torch.save(optimizer.state_dict(), 'result/optimizer.pth')

# Tester
def test():
    network.to(device).eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data = data.to(device)
            target = target.to(device)
            output = network(data)
            test_loss += F.nll_loss(output, target, size_average=False).item()
            pred = output.data.max(1, keepdim=True)[1]
            correct += pred.eq(target.data.view_as(pred)).sum()
        test_loss /= len(test_loader.dataset)
        test_losses.append(test_loss)
        print('\nTest set: Avg. loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
            test_loss, correct, len(test_loader.dataset),
            100. * correct / len(test_loader.dataset)))

In [19]:
network = Net()
optimizer = optim.SGD(network.parameters(), lr=learning_rate,
                      momentum=momentum)
train_losses = []
train_counter = []
test_losses = []
test_counter = [i*len(train_loader.dataset) for i in range(n_epochs + 1)]

In [24]:
summary(network, (1, 28, 28))

Layer (type:depth-idx)                   Output Shape              Param #
├─PureEinConv2d: 1-1                     [-1, 10, 24, 24]          260
├─PureEinConv2d: 1-2                     [-1, 20, 8, 8]            5,020
├─Dropout2d: 1-3                         [-1, 20, 8, 8]            --
├─EinLinear: 1-4                         [-1, 1024]                328,704
├─EinLinear: 1-5                         [-1, 10]                  10,250
Total params: 344,234
Trainable params: 344,234
Non-trainable params: 0
Total mult-adds (M): 0.80
Input size (MB): 0.00
Forward/backward pass size (MB): 0.06
Params size (MB): 1.31
Estimated Total Size (MB): 1.38


Layer (type:depth-idx)                   Output Shape              Param #
├─PureEinConv2d: 1-1                     [-1, 10, 24, 24]          260
├─PureEinConv2d: 1-2                     [-1, 20, 8, 8]            5,020
├─Dropout2d: 1-3                         [-1, 20, 8, 8]            --
├─EinLinear: 1-4                         [-1, 1024]                328,704
├─EinLinear: 1-5                         [-1, 10]                  10,250
Total params: 344,234
Trainable params: 344,234
Non-trainable params: 0
Total mult-adds (M): 0.80
Input size (MB): 0.00
Forward/backward pass size (MB): 0.06
Params size (MB): 1.31
Estimated Total Size (MB): 1.38

In [25]:
test()
total_training_time = 0
for epoch in range(1, n_epochs + 1):
    start = time()
    train(epoch, n_epochs)
    end = time()
    print('\nTraning process takes {} seconds'.format(end - start))
    total_training_time += end - start
    test()

  0%|          | 2/938 [00:00<01:00, 15.50it/s, Loss=2.37]
Test set: Avg. loss: 2.3116, Accuracy: 865/10000 (9%)

Train Epoch 1 of 3
100%|██████████| 938/938 [00:43<00:00, 21.36it/s, Loss=0.555]

Traning process takes 43.93800592422485 seconds
  0%|          | 2/938 [00:00<00:52, 17.85it/s, Loss=0.554]
Test set: Avg. loss: 0.1314, Accuracy: 9595/10000 (96%)

Train Epoch 2 of 3
100%|██████████| 938/938 [00:46<00:00, 20.17it/s, Loss=0.4]

Traning process takes 46.515998125076294 seconds
  0%|          | 2/938 [00:00<00:50, 18.52it/s, Loss=0.399]
Test set: Avg. loss: 0.0852, Accuracy: 9739/10000 (97%)

Train Epoch 3 of 3
100%|██████████| 938/938 [00:47<00:00, 19.88it/s, Loss=0.331]

Traning process takes 47.19996905326843 seconds

Test set: Avg. loss: 0.0692, Accuracy: 9770/10000 (98%)

